## 1: Environment Setup


This notebook synthesizes audio from uploaded MIDI files. There are two different flavors:
* Interpolate between random instruments
* Interpolate between two chosen instruments


Have fun! And please feel free to hack this notebook to make your own creative interactions.

### Instructions for running:

* Make sure to use a GPU runtime, click:  __Runtime >> Change Runtime Type >> GPU__
* Then press the **Play** button on the left of each of the cells
* Double-click any of the cells to view the code



In [ ]:
#@title Install

#@markdown Install magenta, define some helper functions, and download the model. This transfers a lot of data and _should take a minute or two_.

# Install Magenta
print('Copying data from GCS...')
!rm -r /content/gansynth &>/dev/null
!mkdir /content/gansynth
!mkdir /content/gansynth/midi
!mkdir /content/gansynth/samples

# Get default MIDI (Bach Prelude)
!curl -o /content/gansynth/midi/bach.mid http://www.jsbach.net/midi/cs1-1pre.mid
MIDI_SONG_DEFAULT = '/content/gansynth/midi/bach.mid'
!curl -o /content/gansynth/midi/riff-default.mid http://storage.googleapis.com/magentadata/papers/gansynth/midi/arp.mid
MIDI_RIFF_DEFAULT = '/content/gansynth/midi/riff-default.mid'

# All instruments
!curl -o /content/all_instruments.zip https://storage.googleapis.com/magentadata/models/gansynth/all_instruments.zip
!unzip /content/all_instruments.zip

!pip install -q -U magenta

import os
from google.colab import files
import librosa
from magenta.models.nsynth.utils import load_audio
from magenta.models.gansynth.lib import flags as lib_flags
from magenta.models.gansynth.lib import generate_util as gu
from magenta.models.gansynth.lib import model as lib_model
from magenta.models.gansynth.lib import util
import matplotlib.pyplot as plt
import note_seq
from note_seq.notebook_utils import colab_play as play
import numpy as np
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()

# File IO
download = files.download

def upload():
  '''Upload a .wav file.'''
  filemap = files.upload()
  file_list = []
  for key, value in filemap.items():
    fname = os.path.join('/content/gansynth/midi', key)
    with open(fname, 'wb') as f:
      f.write(value)
      print('Writing {}'.format(fname))
    file_list.append(fname)
  return file_list

# GLOBALS
CKPT_DIR = 'gs://magentadata/models/gansynth/acoustic_only'
CKPT_DIR = '/content/all_instruments'

output_dir = '/content/gansynth/samples'
BATCH_SIZE = 16
SR = 16000

# Make an output directory if it doesn't exist
OUTPUT_DIR = util.expand_path(output_dir)
if not tf.gfile.Exists(OUTPUT_DIR):
  tf.gfile.MakeDirs(OUTPUT_DIR)

# Load the model
tf.reset_default_graph()
flags = lib_flags.Flags({
    'batch_size_schedule': [BATCH_SIZE],
    'tfds_data_dir': "gs://tfds-data/datasets",
})
model = lib_model.Model.load_from_path(CKPT_DIR, flags)

# Helper functions
def load_midi(midi_path, min_pitch=36, max_pitch=84):
  """Load midi as a notesequence."""
  midi_path = util.expand_path(midi_path)
  ns = note_seq.midi_file_to_sequence_proto(midi_path)
  pitches = np.array([n.pitch for n in ns.notes])
  velocities = np.array([n.velocity for n in ns.notes])
  start_times = np.array([n.start_time for n in ns.notes])
  end_times = np.array([n.end_time for n in ns.notes])
  valid = np.logical_and(pitches >= min_pitch, pitches <= max_pitch)
  notes = {'pitches': pitches[valid],
           'velocities': velocities[valid],
           'start_times': start_times[valid],
           'end_times': end_times[valid]}
  return ns, notes

def get_envelope(t_note_length, t_attack=0.010, t_release=0.3, sr=16000):
  """Create an attack sustain release amplitude envelope."""
  t_note_length = min(t_note_length, 3.0)
  i_attack = int(sr * t_attack)
  i_sustain = int(sr * t_note_length)
  i_release = int(sr * t_release)
  i_tot = i_sustain + i_release  # attack envelope doesn't add to sound length
  envelope = np.ones(i_tot)
  # Linear attack
  envelope[:i_attack] = np.linspace(0.0, 1.0, i_attack)
  # Linear release
  envelope[i_sustain:i_tot] = np.linspace(1.0, 0.0, i_release)
  return envelope

def combine_notes(audio_notes, start_times, end_times, velocities, sr=16000):
  """Combine audio from multiple notes into a single audio clip.

  Args:
    audio_notes: Array of audio [n_notes, audio_samples].
    start_times: Array of note starts in seconds [n_notes].
    end_times: Array of note ends in seconds [n_notes].
    sr: Integer, sample rate.

  Returns:
    audio_clip: Array of combined audio clip [audio_samples]
  """
  n_notes = len(audio_notes)
  clip_length = end_times.max() + 3.0
  audio_clip = np.zeros(int(clip_length) * sr)

  for t_start, t_end, vel, i in zip(start_times, end_times, velocities, range(n_notes)):
    # Generate an amplitude envelope
    t_note_length = t_end - t_start
    envelope = get_envelope(t_note_length)
    length = len(envelope)
    audio_note = audio_notes[i, :length] * envelope
    # Normalize
    audio_note /= audio_note.max()
    audio_note *= (vel / 127.0)
    # Add to clip buffer
    clip_start = int(t_start * sr)
    clip_end = clip_start + length
    audio_clip[clip_start:clip_end] += audio_note

  # Normalize
  audio_clip /= audio_clip.max()
  audio_clip /= 2.0
  return audio_clip

# Plotting tools
def specplot(audio_clip):
  p_min = np.min(36)
  p_max = np.max(84)
  f_min = librosa.midi_to_hz(p_min)
  f_max = 2 * librosa.midi_to_hz(p_max)
  octaves = int(np.ceil(np.log2(f_max) - np.log2(f_min)))
  bins_per_octave = 36
  n_bins = int(bins_per_octave * octaves)
  C = librosa.cqt(audio_clip, sr=SR, hop_length=2048, fmin=f_min, n_bins=n_bins, bins_per_octave=bins_per_octave)
  power = 10 * np.log10(np.abs(C)**2 + 1e-6)
  plt.matshow(power[::-1, 2:-2], aspect='auto', cmap=plt.cm.magma)
  plt.yticks([])
  plt.xticks([])

print('And...... Done!')

Copying data from GCS...
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  4922  100  4922    0     0  27651      0 --:--:-- --:--:-- --:--:-- 27651
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1083  100  1083    0     0   151k      0 --:--:-- --:--:-- --:--:--  151k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  174M  100  174M    0     0   106M      0  0:00:01  0:00:01 --:--:--  106M
Archive:  /content/all_instruments.zip
replace all_instruments/experiment.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
	adam_beta1: 0.0
	adam_beta2: 0.99
	audio_length: 64000
	batch_size_schedule: [8]
	d_fn: specgram
	data_normalizer: specgrams_prespecified_norma

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/legacy_tf_layers/core.py:332: UserWarning: `tf.layers.flatten` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Flatten` instead.
  warnings.warn('`tf.layers.flatten` is deprecated and '
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
/usr/local/lib/python3.7/dist-packages/magenta/models/gansynth/lib/layers.py:284: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2D` instead.
  kernel_initializer=kernel_initializer)
/usr/local/lib/python3.7/dist-packages/keras/legacy_tf_layers/convolutional.py:575: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call_

INFO:tensorflow:Restoring parameters from /content/all_instruments/stage_00012/./model.ckpt-11000000


INFO:tensorflow:Restoring parameters from /content/all_instruments/stage_00012/./model.ckpt-11000000


And...... Done!


**MY CODE**


1.   Upload midi
2.   Generates N versions of the midi from its pitches
3.   Scores them and chooses the most rock one
4.   



In [ ]:
def upload_midi():

    if True:
      file_list = upload()
      midi_path = file_list[0]
    else:
      midi_path = "/content/anthem.mid"
      #midi_path = "/content/gansynth/midi/bach.mid"
      #midi_path = MIDI_SONG_DEFAULT

    ns, notes = load_midi(midi_path)
    print('Loaded {}'.format(midi_path))
    note_seq.plot_sequence(ns)

    return ns, notes

def generate_n_versions(n, notes):
  
    latent_vectors = []
    generated_audio = []

    for i in range(n):
        # Get random instrument
        if False:
            l = int(notes['end_times'][-1] - 100)
            print(l)
            z_instruments, t_instruments = gu.get_random_instruments(model, notes['end_times'][-1], secs_per_instrument=l)
            print(len(z_instruments), len(t_instruments))
            z_instruments = np.asarray(z_instruments[:1])
            t_instruments = np.asarray([-0.0001, t_instruments[-1]])

            # Get latent vectors for each note
            z_notes = gu.get_z_notes(notes['start_times'], z_instruments, t_instruments)

        z_instrument = model.generate_z(1)
        z_notes = np.repeat(z_instrument, notes['pitches'].shape[0], axis=0)

        # Makes audio samples with latent vector z
        audio_notes = model.generate_samples_from_z(z_notes, notes['pitches'])

        print("Instrument: {}".format(i))
        
        audio_clip = combine_notes(audio_notes,
                                  notes['start_times'],
                                  notes['end_times'],
                                  notes['velocities'], sr=22050)
        play(audio_clip, sample_rate=22050)

        latent_vectors.append(z_instrument)
        generated_audio.append(audio_clip)

    return latent_vectors, generated_audio

def get_rock_score(song):
    pass
    # call model
    # get rock index
    # return

ns, notes = upload_midi()
z_vectors, audio = generate_n_versions(1024, notes)

# Save results
fname = os.path.join(output_dir, f'z_vectors.npy')
np.save(fname, z_vectors)

for i, song in enumerate(audio):
    fname = os.path.join(output_dir, f'generated_clip_{i}.wav')
    gu.save_wav(song, fname)

#rock_scores = [get_rock_score(a) for a in audio]
#i = np.argmax(rock_scores)
#best_z = z_vectors[i]

Saving anthem.mid to anthem (3).mid
Writing /content/gansynth/midi/anthem.mid
Loaded /content/gansynth/midi/anthem.mid


generate_samples: generated 423 samples in 5.781778812408447s
Instrument: 0


generate_samples: generated 423 samples in 5.755282640457153s
Instrument: 1


generate_samples: generated 423 samples in 5.853934288024902s
Instrument: 2


generate_samples: generated 423 samples in 5.9259467124938965s
Instrument: 3


...

generate_samples: generated 423 samples in 5.772327184677124s
Instrument: 1020


generate_samples: generated 423 samples in 5.775115251541138s
Instrument: 1021


generate_samples: generated 423 samples in 5.7796854972839355s
Instrument: 1022


generate_samples: generated 423 samples in 5.777891397476196s
Instrument: 1023


Saved to /content/gansynth/samples/generated_clip_0.wav
Saved to /content/gansynth/samples/generated_clip_1.wav
Saved to /content/gansynth/samples/generated_clip_2.wav
Saved to /content/gansynth/samples/generated_clip_3.wav
Saved to /content/gansynth/samples/generated_clip_4.wav
Saved to /content/gansynth/samples/generated_clip_5.wav
Saved to /content/gansynth/samples/generated_clip_6.wav
Saved to /content/gansynth/samples/generated_clip_7.wav
Saved to /content/gansynth/samples/generated_clip_8.wav
Saved to /content/gansynth/samples/generated_clip_9.wav
Saved to /content/gansynth/samples/generated_clip_10.wav
Saved to /content/gansynth/samples/generated_clip_11.wav
Saved to /content/gansynth/samples/generated_clip_12.wav
Saved to /content/gansynth/samples/generated_clip_13.wav
Saved to /content/gansynth/samples/generated_clip_14.wav
Saved to /content/gansynth/samples/generated_clip_15.wav
Saved to /content/gansynth/samples/generated_clip_16.wav
Saved to /content/gansynth/samples/genera

In [ ]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once in a notebook.
!pip install -U -q PyDrive
!pip install httplib2==0.15.0
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Create & upload a file.
#!zip /content/gansynth/samples.zip -r /content/gansynth/samples
uploaded = drive.CreateFile({'title': 'samples.zip'})
uploaded.SetContentFile('/content/gansynth/samples.zip')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Uploaded file with ID 1BzCIJTK_OJhOKTA9HvRqRlRmyGgSzc9C


In [ ]:
!zip /content/gansynth/samples.zip -r /content/gansynth/samples
download('/content/gansynth/samples.zip')

## Loading optimal latent vectors
We load the list of sampled latent vectors and extract the best one as scored by the music genre classificator.

In [ ]:
import numpy as np

best_idx = {
    'acoustic': [393, 530, 978, 701, 469, 363, 460, 961, 654, 706],
    'all_instruments': [285, 304, 916,  61, 682, 831, 996, 160, 408, 881]
}

model_type = 'acoustic' if 'acoustic' in CKPT_DIR else 'all_instruments'
best_idx = best_idx[model_type][0]

print(f'Upload {model_type} z_vectors')
z_list = upload()
z_list = z_list[0]
z_list = np.load(z_list)  

print('Upload midi file')
midi_path = upload()
midi_path = midi_path[0]

Upload all_instruments z_vectors


Saving z_vectors.npy to z_vectors (3).npy
Writing /content/gansynth/midi/z_vectors.npy
Upload midi file


Saving Donkey Kong 64 - Jungle Japes Central.mid to Donkey Kong 64 - Jungle Japes Central (2).mid
Writing /content/gansynth/midi/Donkey Kong 64 - Jungle Japes Central.mid


We then create audio with GANSynth using a midifile and the chosen latent vector.

In [ ]:
ns, notes = load_midi(midi_path)

z_notes = np.repeat(z_list[best_idx], notes['pitches'].shape[0], axis=0)
audio_notes = model.generate_samples_from_z(z_notes, notes['pitches'])

audio_clip = combine_notes(audio_notes,
                          notes['start_times'],
                          notes['end_times'],
                          notes['velocities'], sr=22050)
play(audio_clip, sample_rate=22050)

fname = f'{model_type}_final.wav'
gu.save_wav(audio_clip, fname)
download(fname)

generate_samples: generated 1724 samples in 1172.318693637848s


Saved to all_instruments_final.wav


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>